In [1]:
import os
import tensorflow as tf
from roboflow import Roboflow
from dotenv import load_dotenv
load_dotenv("labelbox.env")

True

In [2]:
print(tf.version.VERSION)

2.9.1


# - Load Data

---

### --- Load Labelbox Project

In [3]:
import labelbox
LB_API_KEY = os.getenv("labelbox_soccer")
PROJECT_ID = 'clgmw7co50urt070y3evv6o23'
client = labelbox.Client(api_key = LB_API_KEY)
project = client.get_project(PROJECT_ID)
labels = project.export_v2(params={
"data_row_details": True,
"metadata": True,
"attachments": True,
"project_details": True,
"performance_details": True,
"label_details": True,
"interpolated_frames": True
  })

In [4]:
export_json = labels.result
# print("results: ", export_json)

In [ ]:
export_json[0]['data_row']

In [ ]:
export_json[0]['projects']['clgmw7co50urt070y3evv6o23']['labels'][0]['annotations']['classifications']

In [5]:
class_dict = {}
for i in export_json:
    class_dict[i['data_row']['external_id']]= i['projects']['clgmw7co50urt070y3evv6o23']['labels'][0]['annotations']['classifications'][0]['radio_answer']['name']

In [6]:
class_dict

{'480.jpg': 'Yes',
 '482.jpg': 'Yes',
 '484.jpg': 'Yes',
 '486.jpg': 'Yes',
 '488.jpg': 'Yes',
 '490.jpg': 'Yes',
 '492.jpg': 'Yes',
 '494.jpg': 'Yes',
 '496.jpg': 'Yes',
 '498.jpg': 'Yes',
 '500.jpg': 'Yes',
 '502.jpg': 'Yes',
 '504.jpg': 'Yes',
 '506.jpg': 'Yes',
 '508.jpg': 'Yes',
 '510.jpg': 'Yes',
 '512.jpg': 'Yes',
 '514.jpg': 'Yes',
 '516.jpg': 'Yes',
 '518.jpg': 'Yes',
 '520.jpg': 'Yes',
 '522.jpg': 'No',
 '524.jpg': 'Yes',
 '526.jpg': 'Yes',
 '528.jpg': 'Yes',
 '530.jpg': 'Yes',
 '532.jpg': 'Yes',
 '534.jpg': 'No',
 '536.jpg': 'No',
 '538.jpg': 'No',
 '540.jpg': 'Yes',
 '542.jpg': 'Yes',
 '544.jpg': 'Yes',
 '546.jpg': 'Yes',
 '548.jpg': 'Yes',
 '550.jpg': 'Yes',
 '552.jpg': 'Yes',
 '554.jpg': 'Yes',
 '556.jpg': 'Yes',
 '558.jpg': 'Yes',
 '560.jpg': 'Yes',
 '562.jpg': 'Yes',
 '564.jpg': 'Yes',
 '566.jpg': 'Yes',
 '568.jpg': 'No',
 '570.jpg': 'No',
 '572.jpg': 'No',
 '574.jpg': 'Yes',
 '576.jpg': 'Yes',
 '578.jpg': 'Yes',
 '580.jpg': 'Yes',
 '582.jpg': 'Yes',
 '584.jpg': 'Yes',


### --- Load images and classifications

https://financial-engineering.medium.com/tensorflow-2-0-load-images-to-tensorflow-897b8b067fc2

Might not work with tf1: https://notebook.community/tensorflow/docs/site/en/r1/tutorials/load_data/images 

https://keras.io/api/data_loading/image/
https://www.tensorflow.org/tutorials/images/classification

In [7]:
LOCATION = r"C:\Users\yingy\Desktop\Py_personal\Learning\soccer\tf_object_detection\Soccer Object detection.v1i.tfrecord_ignore_data"
# test_record_fname = dataset.location + '/test/cells.tfrecord'
train_record_fname = LOCATION + '/train/people.tfrecord'
raw_dataset = tf.data.TFRecordDataset(train_record_fname)

In [12]:
train_images_dict = {}
for i, raw_record in enumerate(raw_dataset.take(1)):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    file_name=example.features.feature['image/filename'].bytes_list.value[0].decode()
    image_encoded_data = example.features.feature['image/encoded'].bytes_list.value[0]
    train_images_dict[file_name.split("_")[0]+'.jpg'] = image_encoded_data

In [13]:
list(train_images_dict.keys())

['610.jpg']

In [20]:
from PIL import Image
import io
import numpy as np

In [21]:
def process_image_label(image_encoded_data, file_name,class_dict):
    label = True if class_dict[file_name] == 'Yes' else False
    imageStream = io.BytesIO(image_encoded_data)
    img = Image.open(imageStream).convert('RGB')
    image_array = np.array(img)
    image_tensor = tf.convert_to_tensor(image_array, dtype=tf.float32)
    return tf.image.resize(image_tensor, [224,224]), label

In [22]:
image_list = []
label_list = []
for k,v in train_images_dict.items():
    image, label = process_image_label(v,k,class_dict)
    image_list.append(image)
    label_list.append(label)

In [28]:
labeled_dataset = tf.data.Dataset.from_tensor_slices((image_list,label_list))

In [29]:
for image, label in labeled_dataset.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (224, 224, 3)
Label:  True


### --- Load Roboflow Project with image annotation data

In [ ]:
RF_KEY = os.getenv("rb_key")
rf = Roboflow(api_key=RF_KEY)
project = rf.workspace("personal-y96vs").project("soccer-object-detection-yj7hm")
# dataset = project.version(1).download("tensorflow")

In [ ]:
# dataset.location = r"C:\Users\yingy\Desktop\Py_personal\Learning\soccer\tf_object_detection\Soccer Object detection.v1i.tfrecord_ignore_data"
# LOCATION = dataset.location
LOCATION = r"C:\Users\yingy\Desktop\Py_personal\Learning\soccer\tf_object_detection\Soccer Object detection.v1i.tfrecord_ignore_data"
print("location:", LOCATION)
# CLASSES = sorted(project.classes.keys())
# print("classes:", CLASSES)

In [ ]:
# test_record_fname = dataset.location + '/test/cells.tfrecord'
train_record_fname = LOCATION + '/train/people.tfrecord'
label_map_pbtxt_fname = LOCATION + '/train/people_label_map.pbtxt'

In [ ]:
raw_dataset = tf.data.TFRecordDataset(train_record_fname)

In [ ]:
for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
#     print(example.features.feature)
    print(example)

In [ ]:
type(raw_dataset)

# - Plot an image with label

---
Reference: 
https://www.kaggle.com/code/mistag/tensorflow-tfrecords-demystified

In [ ]:
img_example = next(iter(raw_dataset)) 
img_parsed = tf.train.Example.FromString(img_example.numpy())
# only extract features we will actually use
xmin=img_parsed.features.feature['image/object/bbox/xmin'].float_list.value[:]
xmax=img_parsed.features.feature['image/object/bbox/xmax'].float_list.value[:]
ymin=img_parsed.features.feature['image/object/bbox/ymin'].float_list.value[:]
ymax=img_parsed.features.feature['image/object/bbox/ymax'].float_list.value[:]
file_name=img_parsed.features.feature['image/filename'].bytes_list.value[0].decode()
classes=img_parsed.features.feature['image/object/class/text'].bytes_list.value[:]
class_label=img_parsed.features.feature['image/object/class/label'].int64_list.value[:]
img_encoded=img_parsed.features.feature['image/encoded'].bytes_list.value[0]

In [ ]:
file_name

In [ ]:
def bbox(img, xmin, ymin, xmax, ymax, width, label, score):
    draw = ImageDraw.Draw(img)
    xres, yres = img.size[0], img.size[1]
    box = np.multiply([xmin, ymin, xmax, ymax], [xres, yres, xres, yres]).astype(int).tolist()
    txt = " {}: {}%" if score >= 0. else " {}"
    txt = txt.format(label, round(score, 1))
    ts = draw.textbbox((len(txt)*6,8), txt)
    draw.rectangle(box, width=width)
    if len(label) > 0:
        if box[1] >= ts[1]+3:
            xsmin, ysmin = box[0], box[1]-ts[1]-3
            xsmax, ysmax = box[0]+ts[0]+2, box[1]
        else:
            xsmin, ysmin = box[0], box[3]
            xsmax, ysmax = box[0]+ts[0]+2, box[3]+ts[1]+1
        draw.rectangle([xsmin, ysmin, xsmax, ysmax],width = 1)
        draw.text((xsmin, ysmin), txt, fill='white')

def plot_img(img, axes, xmin, ymin, xmax, ymax, classes, class_label):
    for i in range(len(xmin)):
        bbox(img, xmin[i], ymin[i], xmax[i], ymax[i], 2, classes[i].decode(), -1)
    plt.setp(axes, xticks=[], yticks=[])
    plt.imshow(img)

In [ ]:
import numpy as np
from io import BytesIO
from PIL import Image, ImageFont, ImageDraw
fig = plt.figure(figsize=(20,20))
axes = axes = fig.add_subplot(1, 1, 1)
img = Image.open(BytesIO(img_encoded))
plot_img(img, axes, xmin, ymin, xmax, ymax, classes, class_label)

In [ ]:
# Create a dictionary describing the features.
image_feature_description = {
    'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
    'image/object/bbox/ymin':  tf.io.VarLenFeature( tf.float32),
    'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
    'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
    'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    'image/format': tf.io.FixedLenFeature([], tf.string),
    'image/height': tf.io.FixedLenFeature([], tf.int64),
    'image/width': tf.io.FixedLenFeature([], tf.int64),
    'image/filename': tf.io.VarLenFeature(tf.string),
    'image/object/class/text': tf.io.VarLenFeature(tf.string),
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_dataset.map(_parse_image_function)
parsed_image_dataset

In [ ]:
import IPython.display as display
for image_features in parsed_image_dataset.take(1):
    image_raw = image_features['image/encoded'].numpy()
    display.display(display.Image(data=image_raw))

In [ ]:
for i, raw_record in enumerate(raw_dataset.take(1)):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    info = {}
    info['image/encoded'] = example.features.feature['image/encoded'].bytes_list.value[0]

In [ ]:
import io
from PIL import Image

imageStream = io.BytesIO(info['image/encoded'])
imageFile = Image.open(imageStream)
display(imageFile)

# References

https://www.tensorflow.org/guide/data#consuming_tfrecord_data

[How to Train YOLO-NAS on a Custom Dataset](https://blog.roboflow.com/yolo-nas-how-to-train-on-custom-dataset/): [Notebook](https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/train-yolo-nas-on-custom-dataset.ipynb?ref=blog.roboflow.com#scrollTo=sGBOHyPc11LA)

[Read tfrecord dataset for images](https://www.tensorflow.org/tutorials/load_data/tfrecord)